In [1]:
import os
import glob
from tqdm import tqdm

import pandas as pd

In [2]:
root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/datasets/multitask-moe/siim-covid19-detection"

df_study = pd.read_csv(f"{root}/train_study_level.csv")
df_image = pd.read_csv(f"{root}/train_image_level.csv")

df_study

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1
...,...,...,...,...,...
6049,ffcb4630f46f_study,0,1,0,0
6050,ffe4d6e8fbb0_study,0,1,0,0
6051,ffe94fcb14fa_study,0,1,0,0
6052,ffebf1ef4a9c_study,0,1,0,0


In [3]:
df_image

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e
...,...,...,...,...
6329,ffcc6edd9445_image,NaN,none 1 0 0 1 1,7e6c68462e06
6330,ffd91a2c4ca0_image,NaN,none 1 0 0 1 1,8332bdaddb6e
6331,ffd9b6cf2961_image,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814
6332,ffdc682f7680_image,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d


In [4]:
df_study["sum"] = (
    df_study["Negative for Pneumonia"]
    + df_study["Typical Appearance"]
    + df_study["Indeterminate Appearance"]
    + df_study["Atypical Appearance"]
)

df_study["sum"].unique()

array([1])

In [5]:
df_study["StudyInstanceUID"] = df_study["id"].apply(
    lambda x: x.replace("_study", ""))
del df_study["id"]
df_image["ImageUID"] = df_image["id"].apply(lambda x: x.replace("_image", ""))
del df_image["id"]
df_train = df_image.merge(df_study, on="StudyInstanceUID")

df_train = df_train.reset_index(drop=True)

df_train.head()

,boxes,label,StudyInstanceUID,ImageUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,sum
0,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,000a312787f2,0,1,0,0,1
1,NaN,none 1 0 0 1 1,ff0879eb20ed,000c3a3f293f,1,0,0,0,1
2,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,0012ff7358bc,0,1,0,0,1
3,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,001398f4ff4f,0,0,0,1,1
4,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,001bd15d1891,0,1,0,0,1


In [6]:
dict_out = {"id": [], "exam_id": [], "path": [], "class": [], "exam_class": []}

for i in range(len(df_train)):
    item = df_train.iloc[i]

    image_id = item["ImageUID"]
    study_id = item["StudyInstanceUID"]

    potential_files = glob.glob(
        f"{root}/train-png/{study_id}/*/{image_id}.png")

    if len(potential_files) == 0 or len(potential_files) > 1:
        print(study_id)
        continue

    id = f"{study_id}/{potential_files[0].split('/')[-2]}/{image_id}"

    dict_out["id"].append(id)
    dict_out["exam_id"].append(id)
    dict_out["path"].append(f"./train-png/{id}.png")

    if item["Negative for Pneumonia"]:
        cls = "no findings"
    elif item["Typical Appearance"]:
        cls = "typical appearance pneumonia"
    elif item["Indeterminate Appearance"]:
        cls = "indeterminate appearance pneumonia"
    elif item["Atypical Appearance"]:
        cls = "atypical appearance pneumonia"
    else:
        raise NotImplementedError

    dict_out["class"].append(cls)
    dict_out["exam_class"].append(cls)

df_out = pd.DataFrame.from_dict(dict_out)
df_out.head()

,id,exam_id,path,class,exam_class
0,5776db0cec75/81456c9c5423/000a312787f2,5776db0cec75/81456c9c5423/000a312787f2,./train-png/5776db0cec75/81456c9c5423/000a3127...,typical appearance pneumonia,typical appearance pneumonia
1,ff0879eb20ed/d8a644cc4f93/000c3a3f293f,ff0879eb20ed/d8a644cc4f93/000c3a3f293f,./train-png/ff0879eb20ed/d8a644cc4f93/000c3a3f...,no findings,no findings
2,9d514ce429a7/22897cd1daa0/0012ff7358bc,9d514ce429a7/22897cd1daa0/0012ff7358bc,./train-png/9d514ce429a7/22897cd1daa0/0012ff73...,typical appearance pneumonia,typical appearance pneumonia
3,28dddc8559b2/4d47bc042ee6/001398f4ff4f,28dddc8559b2/4d47bc042ee6/001398f4ff4f,./train-png/28dddc8559b2/4d47bc042ee6/001398f4...,atypical appearance pneumonia,atypical appearance pneumonia
4,dfd9fdd85a3e/49170afa4f27/001bd15d1891,dfd9fdd85a3e/49170afa4f27/001bd15d1891,./train-png/dfd9fdd85a3e/49170afa4f27/001bd15d...,typical appearance pneumonia,typical appearance pneumonia


In [7]:
import numpy as np

df_out["dataset"] = "ssim-covid19"
df_out["class_num"] = 4
df_out["contains_negative"] = 1
df_out["modality"] = "xray"

df_out = df_out.sample(frac=1).reset_index(drop=True)
sample_size = len(df_out)

split_array = np.array(["train"] * sample_size)
split_array[-int(sample_size * 0.15):] = "test"
df_out["split"] = split_array

In [8]:
# import numpy as np

# sample_size = len(df_out)

# split_array = np.array(["train"] * sample_size)

# test_indices = np.random.choice(
#     list(range(sample_size)), int(sample_size * 0.2), replace=False)

# split_array[test_indices] = "test"
# df_out["split"] = split_array

In [9]:
df_out["split"].value_counts()

split
train    5384
test      950
Name: count, dtype: int64

In [10]:
df_out.to_csv(f"{root}/metadata.csv", index=False)